# Background

### Z+Jets

In [1]:
import awkward as ak
import pandas as pd
import numpy as np
import vector
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents import NanoAODSchema, DelphesSchema
import mplhep as hep
import matplotlib.pyplot as plt
import seaborn as sns
import DM_HEP_AN as dm
from math import pi
hep.style.use("CMS")
#%matplotlib inline
plt.ioff()

In [4]:
csvFiles = "background/ZpJets/ZpJets_"
nFiles = 20
dfs = [pd.read_csv(csvFiles+str(i)+".csv") for i in range(1,nFiles+1)]
data_z = pd.concat(dfs)
data_z.to_csv("background/ZpJets.csv", index=False)

In [5]:
data_z

,jet_pt0,jet_pt1,jet_pt2,jet_pt3,jet_eta0,jet_eta1,jet_eta2,jet_eta3,jet_phi0,jet_phi1,...,electron_eta2,electron_eta3,electron_phi0,electron_phi1,electron_phi2,electron_phi3,electron_charge0,electron_charge1,electron_charge2,electron_charge3
0,4.098873,3.994257,3.030382,NaN,-0.513310,-0.018580,-4.182301,NaN,2.334446,1.387031,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11.798141,7.737942,6.914221,6.273020,4.573291,1.359862,-0.115720,-5.028740,1.238127,-2.195158,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14.973746,5.966704,5.945081,4.751918,-0.912640,-4.898161,0.937833,-1.075210,-1.969545,0.329583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.759025,4.054801,3.453021,NaN,-4.046280,4.051113,0.217294,NaN,2.776844,2.521697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.678748,NaN,NaN,NaN,1.156869,NaN,NaN,NaN,2.329423,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39912,23.354990,11.897060,10.929173,9.056021,-0.192009,-1.419158,-4.450762,2.683771,0.489044,-0.991664,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39913,12.228477,9.395935,8.234227,7.560238,5.497658,-1.777155,2.990710,4.072493,2.448777,0.226886,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39914,27.299839,20.271896,4.981001,4.693412,-3.291351,3.624152,4.688998,4.011951,-1.511391,2.083552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39915,12.246152,8.501760,7.652135,6.259312,-0.019089,1.271566,1.217181,-2.904069,1.696757,1.723676,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
csvFiles = "background/WpJets/WpJets_"
nFiles = 20
dfs = [pd.read_csv(csvFiles+str(i)+".csv") for i in range(1,nFiles+1)]
data_w = pd.concat(dfs)
data_w.to_csv("background/WpJets.csv", index=False)

In [7]:
data_w

,jet_pt0,jet_pt1,jet_pt2,jet_pt3,jet_eta0,jet_eta1,jet_eta2,jet_eta3,jet_phi0,jet_phi1,...,electron_eta2,electron_eta3,electron_phi0,electron_phi1,electron_phi2,electron_phi3,electron_charge0,electron_charge1,electron_charge2,electron_charge3
0,28.222376,5.638002,4.496588,3.529714,-2.949326,-2.499641,3.079924,-3.498990,2.836737,-1.020999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.699357,6.276242,4.797313,4.376765,-0.514840,-3.350588,-0.899159,0.935917,0.834013,-2.620557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16.280712,14.330416,8.919563,6.994516,-4.552525,2.795676,-1.408053,0.593003,2.099661,-1.320631,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27.970625,6.510084,5.437975,4.527232,-0.340904,1.009112,1.125909,-4.168434,-1.481624,3.026858,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26.476050,6.379861,3.833766,3.722295,2.709292,-0.134858,1.497421,0.844396,-0.717510,-2.960393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41331,24.158780,20.411522,15.632806,12.163850,1.539346,0.733054,-2.485925,-4.222438,1.221279,-1.769047,...,NaN,NaN,-1.454934,NaN,NaN,NaN,1.0,NaN,NaN,NaN
41332,21.796290,3.472583,NaN,NaN,1.707460,-0.851663,NaN,NaN,1.822857,-1.380249,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41333,19.466852,17.616493,8.940805,6.488828,-0.344921,0.062187,-3.861596,-1.809226,-1.136709,1.097524,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41334,9.398885,9.229892,7.806156,6.892436,0.474567,-3.215132,2.649302,3.661700,2.447900,-0.199796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
